## Implementacja metody UTAstar dyskretnej

In [23]:
import numpy as np
from scipy.interpolate import interp1d

# Funkcja do wyznaczania funkcji użyteczności dla wersji dyskretnej
def UTA_function_discrete(values, reference_points, lambda_param=1):
    """
    Aproksymuje funkcje użyteczności dla wersji dyskretnej.
    
    :param values: Lista wartości alternatyw w przestrzeni kryteriów
    :param reference_points: Lista punktów referencyjnych (najlepszy, najgorszy, neutralny)
    :param lambda_param: Współczynnik λ (preferencje decydenta)
    :return: Lista funkcji użyteczności dla wszystkich alternatyw
    """
    utility_values = []
    
    # Iterujemy przez alternatywy
    for val in values:
        alternative_utility = []
        
        # Iterujemy przez każde kryterium
        for i in range(len(val)):
            ref_values = [ref[i] for ref in reference_points]  # Zbiór punktów referencyjnych dla kryterium i
            # Interpolacja liniowa dla każdego kryterium
            interp_func = interp1d(ref_values, [-1, 0, 1], kind='linear', fill_value="extrapolate")
            utility_value = interp_func(val[i]) * lambda_param  # Użycie λ
            alternative_utility.append(utility_value)
        
        utility_values.append(alternative_utility)
    
    return np.array(utility_values)

# Funkcja do porównania alternatyw (dyskretna wersja)
def UTAstar_discrete(values, reference_points, lambda_param=1):
    """
    Metoda UTA Star dla wersji dyskretnej (wielokryterialnej) uwzględniająca preferencje decydenta.
    
    :param values: Lista wartości alternatyw w przestrzeni kryteriów
    :param reference_points: Lista punktów referencyjnych dla każdego kryterium
    :param lambda_param: Współczynnik λ (preferencje decydenta)
    :return: Wybrana alternatywa
    """
    # Wyznaczanie funkcji użyteczności
    utility_values = UTA_function_discrete(values, reference_points, lambda_param)
    
    # Wybór rozwiązania kompromisowego - wybieramy alternatywę z maksymalną sumą użyteczności
    summed_utility = np.sum(utility_values, axis=1)
    best_solution_idx = np.argmax(summed_utility)  # Wybieramy alternatywę z największą sumą funkcji użyteczności
    return best_solution_idx, utility_values

# Przykład użycia (wersja dyskretna)
values_discrete = np.array([[20, 30], [50, 60], [80, 90], [35, 45], [65, 70]])  # Przykładowe wartości alternatyw
reference_points_discrete = np.array([[20, 30], [80, 90], [50, 60]])  # Punkty referencyjne (najlepszy, najgorszy, neutralny)
lambda_param = 1.5  # Przykładowy współczynnik λ (preferencje decydenta)
best_solution_discrete_idx, utility_values_discrete = UTAstar_discrete(values_discrete, reference_points_discrete, lambda_param=lambda_param)

print(f"Najlepsza alternatywa (dyskretna): {values_discrete[best_solution_discrete_idx]} z funkcją użyteczności {utility_values_discrete[best_solution_discrete_idx]}")


Najlepsza alternatywa (dyskretna): [50 60] z funkcją użyteczności [1.5 1.5]


## Implementacja UTAstar ciągłej

In [24]:
import numpy as np
from scipy.interpolate import interp1d
from scipy.optimize import minimize

# Funkcja do wyznaczania funkcji użyteczności dla wersji ciągłej
def UTA_function_continuous(values, reference_points, lambda_param=1, use_polynomial=False):
    """
    Aproksymuje funkcje użyteczności dla wersji ciągłej.

    :param values: Lista wartości alternatyw w przestrzeni kryteriów (ciągłe wartości)
    :param reference_points: Lista punktów referencyjnych (najlepszy, najgorszy, neutralny)
    :param lambda_param: Współczynnik λ (preferencje decydenta)
    :param use_polynomial: Flaga, czy używać interpolacji wielomianowej zamiast liniowej
    :return: Lista funkcji użyteczności dla wszystkich alternatyw
    """
    utility_functions = []

    # Budowa funkcji użyteczności dla każdego kryterium
    for i in range(len(reference_points[0])):
        ref_values = [ref[i] for ref in reference_points]  # Punkty referencyjne dla kryterium i
        utility_values = [-1, 0, 1]  # Wartości użyteczności odpowiadające punktom referencyjnym

        if use_polynomial:
            # Interpolacja wielomianowa za pomocą np.polyfit
            coeffs = np.polyfit(ref_values, utility_values, deg=min(len(ref_values)-1, 2))
            def poly_func(x, coeffs=coeffs):
                return np.polyval(coeffs, x)
            utility_functions.append(poly_func)
        else:
            # Tworzenie funkcji aproksymującej (ciągła interpolacja liniowa)
            interp_func = interp1d(ref_values, utility_values, kind='linear', fill_value="extrapolate")
            utility_functions.append(interp_func)

    # Wyznaczanie wartości funkcji użyteczności dla alternatyw
    utility_values = []
    for val in values:
        alternative_utility = []
        for i in range(len(val)):
            utility_value = utility_functions[i](val[i]) * lambda_param  # Obliczanie użyteczności dla kryterium i
            alternative_utility.append(utility_value)
        utility_values.append(alternative_utility)

    return np.array(utility_values)

# Funkcja optymalizacji w wersji ciągłej
def UTAstar_continuous(values, reference_points, lambda_param=1, use_polynomial=False):
    """
    Metoda UTA Star dla wersji ciągłej (wielokryterialnej).

    :param values: Lista wartości alternatyw w przestrzeni kryteriów
    :param reference_points: Lista punktów referencyjnych dla każdego kryterium
    :param lambda_param: Współczynnik λ (preferencje decydenta)
    :param use_polynomial: Flaga, czy używać interpolacji wielomianowej zamiast liniowej
    :return: Najlepsza alternatywa oraz jej użyteczność
    """
    # Wyznaczanie bounds na podstawie wartości alternatyw
    bounds = [(np.min(values[:, i]), np.max(values[:, i])) for i in range(values.shape[1])]

    # Funkcja celu: Maksymalizacja sumy użyteczności
    def objective_function(x):
        utility_values = UTA_function_continuous([x], reference_points, lambda_param=lambda_param, use_polynomial=use_polynomial)
        return -np.sum(utility_values)  # Negujemy, ponieważ minimalizujemy w optymalizacji

    # Początkowy punkt startowy (środek przedziałów)
    initial_guess = [(b[0] + b[1]) / 2 for b in bounds]

    # Ograniczenia w formacie scipy.optimize
    constraints = [(b[0], b[1]) for b in bounds]

    # Optymalizacja
    result = minimize(objective_function, initial_guess, bounds=constraints, method='SLSQP')

    # Wartości użyteczności dla punktów wejściowych
    utility_values = objective_function(values)

    if result.success:
        best_solution = result.x
        best_utility = -result.fun  # Odpowiada maksymalnej użyteczności
        best_utility_vals = UTA_function_continuous([best_solution], reference_points, lambda_param=lambda_param, use_polynomial=use_polynomial)
        return best_solution, best_utility, utility_values, best_utility_vals
    else:
        raise ValueError("Optymalizacja nie powiodła się.")


# Przykład użycia (wersja ciągła)
values_discrete = np.array([
    [10, 20, 30, 60],
    [20, 30, 40, 50],
    [50, 60, 70, 80],
    [80, 90, 100, 110],
    [35, 45, 55, 65],
    [65, 75, 85, 95]
])

# Punkty referencyjne (najlepszy, najgorszy, neutralny) w 4 wymiarach
reference_points_continuous = np.array([
    [80, 90, 100, 10],  # Najgorszy
    [50, 60, 70, 80],   # Neutralny
    [20, 30, 40, 55],  # Najlepszy
])

# values_discrete = np.array([[20, 30], [50, 60], [80, 90], [35, 45], [65, 70]])  # Przykładowe wartości alternatyw
# reference_points_continuous = np.array([[20, 30], [80, 90], [50, 60]])  # Punkty referencyjne (najlepszy, najgorszy, neutralny)
lambda_param = 1  # Przykładowy współczynnik λ (preferencje decydenta)

best_solution_continuous, best_utility_continuous_sum, best_utility_continuous_vals = UTAstar_continuous(values_discrete, reference_points_continuous, lambda_param=lambda_param, use_polynomial=True)

print(f"Najlepsza alternatywa (ciągła): {best_solution_continuous} z użytecznością {best_utility_continuous_sum} | {best_utility_continuous_vals}")


Najlepsza alternatywa (ciągła): [10.         20.         30.         50.92106426] z użytecznością 5.020071010860321 | [[1.33333333 1.33333333 1.33333333 1.02007101]]


# Testy obliczeniowe

## Testy Dyskretne

### Dyskretne 3-wymiarowe

In [25]:
# Przykładowe wartości alternatyw w 3 wymiarach (3 kryteria)
values_discrete_3d = np.array([
    [20, 30, 40],
    [50, 60, 70],
    [80, 90, 100],
    [35, 45, 55],
    [65, 75, 85]
])

# Punkty referencyjne (najlepszy, najgorszy, neutralny) w 3 wymiarach
reference_points_discrete_3d = np.array([
    [20, 30, 40],  # Najgorszy
    [50, 60, 70],   # Neutralny
    [80, 90, 100],  # Najlepszy
])

lambda_param = 1.5  # Współczynnik λ (preferencje decydenta)

# Wywołanie metody UTA Star dla wersji dyskretnej
best_solution_discrete_3d_idx, utility_values_discrete_3d = UTAstar_discrete(values_discrete_3d, reference_points_discrete_3d, lambda_param=lambda_param)

print(f"Najlepsza alternatywa (dyskretna, 3D): {values_discrete_3d[best_solution_discrete_3d_idx]} z funkcją użyteczności {utility_values_discrete_3d[best_solution_discrete_3d_idx]}")

Najlepsza alternatywa (dyskretna, 3D): [ 80  90 100] z funkcją użyteczności [1.5 1.5 1.5]


### Dyskretne 4-wymiarowe

In [26]:
# Przykładowe wartości alternatyw w 4 wymiarach (4 kryteria)
values_discrete_4d = np.array([
    [20, 30, 40, 50],
    [50, 60, 70, 80],
    [80, 90, 100, 110],
    [35, 45, 55, 65],
    [65, 75, 85, 95]
])

# Punkty referencyjne (najlepszy, najgorszy, neutralny) w 4 wymiarach
reference_points_discrete_4d = np.array([
    [80, 90, 100, 110],  # Najgorszy
    [50, 60, 70, 80],   # Neutralny
    [20, 30, 40, 50],  # Najlepszy
])

lambda_param = 1.5  # Współczynnik λ (preferencje decydenta)

# Wywołanie metody UTA Star dla wersji dyskretnej
best_solution_discrete_4d_idx, utility_values_discrete_4d = UTAstar_discrete(values_discrete_4d, reference_points_discrete_4d, lambda_param=lambda_param)

print(f"Najlepsza alternatywa (dyskretna, 4D): {values_discrete_4d[best_solution_discrete_4d_idx]} z funkcją użyteczności {utility_values_discrete_4d[best_solution_discrete_4d_idx]}")


Najlepsza alternatywa (dyskretna, 4D): [20 30 40 50] z funkcją użyteczności [1.5 1.5 1.5 1.5]


## Testy Ciągłe

### Ciągłe 3-wymiarowe

In [27]:
# Przykładowe wartości alternatyw w 3 wymiarach (3 kryteria) w przestrzeni ciągłej
values_continuous_3d = np.array([
    [20, 30, 40],
    [50, 60, 70],
    [80, 90, 100],
    [35, 45, 55],
    [65, 75, 85]
])

# Punkty referencyjne (najlepszy, najgorszy, neutralny) w 3 wymiarach
reference_points_continuous_3d = np.array([
    [20, 30, 40],  # Najgorszy
    [50, 60, 100],   # Neutralny
    [80, 90, 70],  # Najlepszy
])

lambda_param = 1.5  # Współczynnik λ (preferencje decydenta)

# Wywołanie metody UTA Star dla wersji ciągłej
best_solution_continuous, best_utility_continuous_sum, best_utility_continuous_vals = UTAstar_continuous(values_continuous_3d, reference_points_continuous_3d, lambda_param=lambda_param, use_polynomial=False)

print(f"Najlepsza alternatywa (ciągła): {best_solution_continuous} z użytecznością {best_utility_continuous_sum} | {best_utility_continuous_vals}")


Najlepsza alternatywa (ciągła): [50.00004883 60.00004883 69.99995117] z użytecznością 1.5000000000000002 | [[2.44140610e-06 2.44140610e-06 1.49999512e+00]]


In [28]:
# Przykładowe wartości alternatyw w 3 wymiarach (3 kryteria) w przestrzeni ciągłej
values_continuous_3d = np.array([
    [20, 30, 40],
    [50, 60, 70],
    [80, 90, 100],
    [35, 45, 55],
    [65, 75, 85]
])

# Punkty referencyjne (najlepszy, najgorszy, neutralny) w 3 wymiarach
reference_points_continuous_3d = np.array([
    [20, 30, 40],  # Najgorszy
    [50, 60, 100],   # Neutralny
    [80, 90, 70],  # Najlepszy
])

lambda_param = 1.5  # Współczynnik λ (preferencje decydenta)

# Wywołanie metody UTA Star dla wersji ciągłej
best_solution_continuous, best_utility_continuous_sum, best_utility_continuous_vals = UTAstar_continuous(values_continuous_3d, reference_points_continuous_3d, lambda_param=lambda_param, use_polynomial=True)

print(f"Najlepsza alternatywa (ciągła): {best_solution_continuous} z użytecznością {best_utility_continuous_sum} | {best_utility_continuous_vals}")


Najlepsza alternatywa (ciągła): [80.         90.         74.99998288] z użytecznością 4.562499999999267 | [[1.5    1.5    1.5625]]


### Ciągłe 4-wymiarowe

In [29]:
# Przykładowe wartości alternatyw w 4 wymiarach (4 kryteria) w przestrzeni ciągłej
values_continuous_4d = np.array([
    [20, 30, 40, 50],
    [50, 60, 70, 80],
    [80, 90, 100, 110],
    [35, 45, 55, 65],
    [65, 75, 85, 95]
])

# Punkty referencyjne (najlepszy, najgorszy, neutralny) w 4 wymiarach
reference_points_continuous_4d = np.array([
    [80, 90, 100, 110],  # Najgorszy
    [50, 60, 70, 50],   # Neutralny
    [20, 30, 40, 80],  # Najlepszy
])

lambda_param = 1.5  # Współczynnik λ (preferencje decydenta)

# Wywołanie metody UTA Star dla wersji ciągłej
best_solution_continuous, best_utility_continuous_sum, best_utility_continuous_vals = UTAstar_continuous(values_continuous_4d, reference_points_continuous_4d, lambda_param=lambda_param, use_polynomial=False)

print(f"Najlepsza alternatywa (ciągła): {best_solution_continuous} z użytecznością {best_utility_continuous_sum} | {best_utility_continuous_vals}")


Najlepsza alternatywa (ciągła): [20.         30.         40.         79.99999548] z użytecznością 5.999999774108704 | [[1.5        1.5        1.5        1.49999977]]


In [30]:
# Przykładowe wartości alternatyw w 4 wymiarach (4 kryteria) w przestrzeni ciągłej
values_continuous_4d = np.array([
    [20, 30, 40, 50],
    [50, 60, 70, 80],
    [80, 90, 100, 110],
    [35, 45, 55, 65],
    [65, 75, 85, 95]
])

# Punkty referencyjne (najlepszy, najgorszy, neutralny) w 4 wymiarach
reference_points_continuous_4d = np.array([
    [80, 90, 100, 110],  # Najgorszy
    [50, 60, 70, 50],   # Neutralny
    [20, 30, 40, 80],  # Najlepszy
])

lambda_param = 1.5  # Współczynnik λ (preferencje decydenta)

# Wywołanie metody UTA Star dla wersji ciągłej
best_solution_continuous, best_utility_continuous_sum, best_utility_continuous_vals = UTAstar_continuous(values_continuous_4d, reference_points_continuous_4d, lambda_param=lambda_param, use_polynomial=True)

print(f"Najlepsza alternatywa (ciągła): {best_solution_continuous} z użytecznością {best_utility_continuous_sum} | {best_utility_continuous_vals}")


Najlepsza alternatywa (ciągła): [20.         30.         40.         75.00001196] z użytecznością 6.062499999999634 | [[1.5    1.5    1.5    1.5625]]


In [3]:
import numpy as np 
def prepare_matrix_for_fuzzy_topsis(alternatives_vector, lambda_param=1):
    """
    Przygotowuje macierz alternatyw do metody TOPSIS z wykorzystaniem rozmytych liczb.
    
    :param alternatives_vector: Wektor alternatyw (macierz alternatyw)
    :return: Przygotowana macierz alternatyw

    Example:
    - input:
    np.array[[x1, x2, x3],
     [y1, y2, y3],
     [z1, z2, z3]]
    - output
    np.array[[(x1-lambda, x1, x1+lambda), (x2-lambda, x2, x2+lambda), (x3-lambda, x3, x3+lambda)],
     [(y1-lambda, y1, y1+lambda), (y2-lambda, y2, y2+lambda), (y3-lambda, y3, y3+lambda)],
     [(z1-lambda, z1, z1+lambda), (z2-lambda, z2, z2+lambda), (z3-lambda, z3, z3+lambda)]]
    """
    # Wyznaczenie rozmiaru macierzy
    n, m = alternatives_vector.shape

    # Przygotowanie macierzy wynikowej
    fuzzy_matrix = np.zeros((n, m, 3))

    # Iteracja po alternatywach
    for i in range(n):
        # Iteracja po kryteriach
        for j in range(m):
            # Wyznaczenie wartości rozmytej
            fuzzy_value = (alternatives_vector[i, j] - lambda_param, alternatives_vector[i, j], alternatives_vector[i, j] + lambda_param)
            fuzzy_matrix[i, j] = fuzzy_value

    return fuzzy_matrix

temp = np.array([
    [20, 30, 40, 50],
    [50, 60, 70, 80],
    [80, 90, 100, 110],
    [35, 45, 55, 65],
    [65, 75, 85, 95]
])

print(temp)
print(prepare_matrix_for_fuzzy_topsis(temp, 1.5))

[[ 20  30  40  50]
 [ 50  60  70  80]
 [ 80  90 100 110]
 [ 35  45  55  65]
 [ 65  75  85  95]]
[[[ 18.5  20.   21.5]
  [ 28.5  30.   31.5]
  [ 38.5  40.   41.5]
  [ 48.5  50.   51.5]]

 [[ 48.5  50.   51.5]
  [ 58.5  60.   61.5]
  [ 68.5  70.   71.5]
  [ 78.5  80.   81.5]]

 [[ 78.5  80.   81.5]
  [ 88.5  90.   91.5]
  [ 98.5 100.  101.5]
  [108.5 110.  111.5]]

 [[ 33.5  35.   36.5]
  [ 43.5  45.   46.5]
  [ 53.5  55.   56.5]
  [ 63.5  65.   66.5]]

 [[ 63.5  65.   66.5]
  [ 73.5  75.   76.5]
  [ 83.5  85.   86.5]
  [ 93.5  95.   96.5]]]
